Let's first set up the python environment

In [2]:
# Import required modules
import shutil
import sys
from pathlib import Path

import numpy as np

# Import required icepy4d4D modules
from icepy4d import classes as icepy4d_classes
from icepy4d import matching
from icepy4d import sfm
from icepy4d import io
from icepy4d import utils as icepy4d_utils
from icepy4d.classes.epoch import Epoch, Epoches
from icepy4d.metashape import metashape as MS
from icepy4d.utils import initialization as inizialization

Define the configuration file

In [3]:
# Define the path to the configuration file
CFG_FILE = "config/config_2022.yaml"

Inizialize all the required variables

In [4]:
# Parse the configuration file
cfg_file = Path(CFG_FILE)
cfg = inizialization.parse_cfg(cfg_file)

# Initialize the timer and logger
timer_global = icepy4d_utils.AverageTimer()
logger = icepy4d_utils.get_logger()

# Get the list of cameras from the configuration file
cams = cfg.cams

# Get the list of images from the configuration file
images, epoch_dict = inizialization.initialize_image_ds(cfg)

# Initialize an empty Epoches object to store the results of each epoch
epoches = Epoches(starting_epoch=cfg.proc.epoch_to_process[0])


ICEpy4D
Image-based Continuos monitoring of glaciers' Evolution with low-cost stereo-cameras and Deep Learning photogrammetry
2023 - Francesco Ioli - francesco.ioli@polimi.it

2023-08-29 14:52:42 | [INFO    ] Configuration file: config_2022
2023-08-29 14:52:42 | [INFO    ] Epoch_to_process set to a pair of values. Expanding it for a range of epoches from epoch 0 to 158.
2023-08-29 14:52:42 | [INFO    ] Image datastores created successfully.
